<a href="https://colab.research.google.com/github/HebaRouk/finetunedAraGpt_UIGradio.ipynb/blob/main/finetunedAraGpt_UIGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparation

In [ ]:
#drive.flush_and_unmount()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ln -s drive/MyDrive/Aragpt40Epochs/ ./Ara
#!ln -s drive/MyDrive/Colab\ Notebooks/Aragpt40Epochs/ ./Ara

In [ ]:
#!pip install gpt-2-simple==0.7.1

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install Gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
#!unzip aragptFineTuned40EP.zip

In [ ]:
#!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
import torch, gc

#Load the model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
import os

#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# Load the fine-tuned model and tokenizer
model_path = "./Ara"  # Update this path if needed
#model_path = "aubmindlab/aragpt2-base"  # Update this path if needed

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
#model = model.to('cuda')


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)


# Set the pad token to be the same as eos_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#Feature 1:  Chapter by chapter generation

In [ ]:
output_chapters =[]

#Previous_chapters,
# Function to generate a chapter based on user input
def generate_first_chapter( topic):#, chapter2, chapter3):
    global output_chapters
    output_chapters =[]
    print(len(output_chapters))
    #topic = chapter1  # Using Chapter 1 input as topic
    #print("$$$$$$$$$$$$$")
    sys_prompt = "أنت كاتب مبدع وخبير في أدب الأطفال والناشئة. مختص في إنشاء الفصل التالي .يجب ان يحتوي الفصل التالي على 500 كلمة كحد أدنى"
    usr_prompt = f"قم بإنشاء الفصل الاول استنادا فقط الى الفكرة الرئيسية: الفكرة الرئيسية للقصة هي {topic}.\n"
    prompt = f"{sys_prompt}\n{usr_prompt}"

    # Tokenize with padding and return attention_mask
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
    #encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    input_ids = encoded_input.input_ids
    attention_mask = encoded_input.attention_mask
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Generate text with the attention mask explicitly provided
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,        # Adjust this if you expect longer output
        do_sample=True,        # Enable sampling for more creative text
        top_k=50,              # Top-K sampling
        top_p=0.95,            # Nucleus sampling
        num_return_sequences=1, # Generate one sequence
        temperature=0.0001,
    )
    torch.cuda.empty_cache()
    gc.collect()

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Decode the generated tokens to text
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Return generated chapter content
    #return f"### Chapter 1: {generated_text}"
    new_chapter = generated_text[len(prompt):]
    output_chapters.append(new_chapter)
    # Print the generated chapter content
    print(len(output_chapters))
    torch.cuda.empty_cache()
    gc.collect()
    return new_chapter

def generate_next_chapter(topic, chapter_number):#, chapter2, chapter3):
    print("\n########### CHAPTER", chapter_number, "#############################\n")
    global output_chapters
    #topic = chapter1  # Using Chapter 1 input as topic
    output_chapters = output_chapters[:int(chapter_number)-1]
    print(len(output_chapters))
    #previous_chapters = "\n\n".join(output_chapters[:int(chapter_number)-1])
    previous_chapters = "\n\n".join(output_chapters)
    print("**** Previous chapters ****\n", previous_chapters,'\n')
    #previous_chapters = "\n\n".join(output_chapters[:1])
    sys_prompt = "أنت كاتب مبدع وخبير في أدب الأطفال والناشئة. مختص في إنشاء الفصل التالي .يجب ان يحتوي الفصل التالي على 500 كلمة كحد أدنى"
    #usr_prompt = f"قم بإنشاء الفصل الاول استنادا فقط الى الفكرة الرئيسية: الفكرة الرئيسية للقصة هي {topic}.\n"

    usr_prompt = (
            f"باستخدام الفصول السابقة:\n{previous_chapters}\n\n"
            f"وبناءً على الفكرة الرئيسية للفصل {chapter_number}: '{topic}', قم بإنشاء الفصل {chapter_number}.\n"
        )
    prompt = f"{sys_prompt}\n{usr_prompt}"

    # Tokenize with padding and return attention_mask
    #encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=200)
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    input_ids = encoded_input.input_ids
    attention_mask = encoded_input.attention_mask
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Generate text with the attention mask explicitly provided
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128 * int(chapter_number),        # Adjust this if you expect longer output
        do_sample=True,        # Enable sampling for more creative text
        top_k=50,              # Top-K sampling
        top_p=0.95,            # Nucleus sampling
        num_return_sequences=1, # Generate one sequence
        temperature=0.0001,
    )
    torch.cuda.empty_cache()
    gc.collect()

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Decode the generated tokens to text
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Return generated chapter content
    #return f"### Chapter 1: {generated_text}"
    new_chapter = generated_text[len(prompt):]
    output_chapters.append(new_chapter)
    print("******** Chapter", chapter_number, "*********\n", new_chapter, '\n')
    print(len(output_chapters))
    torch.cuda.empty_cache()
    gc.collect()
    return new_chapter


# Create Gradio interface for user input
with gr.Blocks() as demo:
    # Header or title for the interface
    gr.Markdown("# مولد القصة التفاعلية ")

    # Instructions or subheader
    gr.Markdown("اكتب فكرتك لكل فصل")

    with gr.Tabs():  # Using tabs as shown in your design

        with gr.TabItem("الفصل 1"):
            #chapter1_input = gr.Textbox(label="Chapter 1 Idea", placeholder="Describe Chapter 1 here...", lines=2)
            #chapter1_input = gr.Textbox("فتاة فقيرة تسمى ميساء تعيش مع عمها و زوجته الشريرة", label="Chapter 1 Idea", placeholder="Describe Chapter 1 here...", lines=2)
            chapter1_input = gr.Textbox("يعيش احمد في منزل صغير في الصحراء", label="فكرةالفصل 1", placeholder="Describe الفصل 1 here...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("انشاء الفصل 1")
            # Output area to display the generated content
            output = gr.Textbox(label="انشاء الفصل 1", interactive=False, lines=10)
            # Button click action: Capture inputs and display the generated story
            submit_button.click(generate_first_chapter, inputs= chapter1_input, outputs=output)


        with gr.TabItem("الفصل 2"):
            #chapter_input = gr.Textbox("اروي الصعوبات الي تواجه ميساء نفسيا و ماديا و اسريا و التحديات الي تواجهها", label="Chapter 2 Idea", placeholder="Describe Chapter 2 here...", lines=2)
            chapter_input = gr.Textbox("عثور احمد على خريطه صدفه", label=" 2 فكرةالفصل ", placeholder="Describe الفصل 2 here...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("انشاء الفصل 2")

            # Output area to display the generated content
            output = gr.Textbox(label="انشاء الفصل 2", interactive=False, lines=2)
            #print("-------------", output_chapters[0])
            # Button click action: Capture inputs and display the generated story
            #previous_chapters = "\n\n".join(output_chapters[:1])
            submit_button.click(generate_next_chapter, inputs= [chapter_input, gr.State('2')], outputs=output)

        with gr.TabItem("الفصل 3"):
            #chapter_input = gr.Textbox("وجدت ميساء كنزا في الحديقة و كيف تغير حالها و كيف انتقمت من زوجة عمها", label="Chapter 3 Idea", placeholder="Describe Chapter 3 here...", lines=2)
            chapter_input = gr.Textbox("الصعوبات التي واجهته", label="فكرةالفصل 3 ", placeholder="Describe الفصل 3 here...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("انشاء الفصل 3")
            # Output area to display the generated content
            output = gr.Textbox(label="انشاء الفصل 3", interactive=False, lines=2)
            #print("-------------", output_chapters[0])
            # Button click action: Capture inputs and display the generated story
            #previous_chapters = "\n\n".join(output_chapters[:1])
            submit_button.click(generate_next_chapter, inputs= [chapter_input, gr.State('3')], outputs=output)

        with gr.TabItem("الفصل 4"):
            #chapter_input = gr.Textbox("وجدت ميساء كنزا في الحديقة و كيف تغير حالها و كيف انتقمت من زوجة عمها", label="Chapter 3 Idea", placeholder="Describe Chapter 4 here...", lines=2)
            chapter_input = gr.Textbox("الملاك الذي انقذ احمد", label="فكرةالفصل 4 ", placeholder="Describe الفصل 4 here...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("انشاء الفصل  4 ")
            # Output area to display the generated content
            output = gr.Textbox(label="انشاء الفصل  4", interactive=False, lines=2)
            #print("-------------", output_chapters[0])
            # Button click action: Capture inputs and display the generated story
            #previous_chapters = "\n\n".join(output_chapters[:1])
            submit_button.click(generate_next_chapter, inputs= [chapter_input, gr.State('4')], outputs=output)
        with gr.TabItem("الفصل 5"):
            chapter_input = gr.Textbox("عوده احمد الى المنزل بسلام يروي قصته", label="فكرةالفصل 5 ", placeholder="Describe الفصل 5 here...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("انشاء الفصل 5 ")
            # Output area to display the generated content
            output = gr.Textbox(label="انشاء الفصل 5", interactive=False, lines=2)
            #print("-------------", output_chapters[0])
            # Button click action: Capture inputs and display the generated story
            #previous_chapters = "\n\n".join(output_chapters[:1])
            submit_button.click(generate_next_chapter, inputs= [chapter_input, gr.State('5')], outputs=output)

# Launch the interface
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://38b8c4c3ffeb553578.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


0
1


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



########### CHAPTER 2 #############################

1
**** Previous chapters ****
 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب� 

******** Chapter 2 *********
 عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب 

2


...


########### CHAPTER 3 #############################

2
**** Previous chapters ****
 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب�

عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب 

******** Chapter 3 *********
 بعد أن وافق أحمد وعلي على الانضمام إلى أحمد في الرحلة، بدأا في تجهيز الأدوات اللازمة للرحلة. كان عليهما جمع بعض الماء والطعام والبوصلة وخريطة قديمة وجدها أحمد في مكتبة جده.

في صباح اليوم التالي، انطلقا في طريقهما نحو الصحراء. كان الهواء 

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



########### CHAPTER 4 #############################

3
**** Previous chapters ****
 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب�

عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب

بعد أن وافق أحمد وعلي على الانضمام إلى أحمد في الرحلة، بدأا في تجهيز الأدوات اللازمة للرحلة. كان عليهما جمع بعض الماء والطعام والبوصلة وخريطة قديمة وجدها أحمد في مكتبة جده.

في صباح اليوم التالي، انطلقا في طريقهما نحو الصحراء. كان الهواء منعشًا والشمس تشرق بلطف، مما أض

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



########### CHAPTER 5 #############################

4
**** Previous chapters ****
 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب�

عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب

بعد أن وافق أحمد وعلي على الانضمام إلى أحمد في الرحلة، بدأا في تجهيز الأدوات اللازمة للرحلة. كان عليهما جمع بعض الماء والطعام والبوصلة وخريطة قديمة وجدها أحمد في مكتبة جده.

في صباح اليوم التالي، انطلقا في طريقهما نحو الصحراء. كان الهواء منعشًا والشمس تشرق بلطف، مما أض

In [ ]:
print(generate_next_chapter())

#Feature 2: Question Answering

In [ ]:
######"child_question": {
#            "role": "user",
#            "content": "ما هو السر وراء لمعان ريش عصفور الشمس؟"
#        },
#        "chatbot_answer": {
#            "role": "assistant",
#            "content": "أهلاً بك! لمعان ريش عصفور الشمس يأتي من تميزه وجماله الفريد. يقال إن هذا العصفور يمتلك ريشًا مصنوعًا من الذهب، وهو ما يجعله يبدو مشعًا ومتألقًا تحت أشعة الشمس. اللمعان الذي يظهر من ريشه يعكس جماله الد"
#        }

In [ ]:
story = """### القصة:
في قرية صغيرة محاطة بأشجار مورقة، عاش العصفور الصغير شموس بريشٍ ذهبي لامع. ذات صباحٍ مشرق، انضمت إليهم العصفورة الملونة درة، واقترحوا معًا الانطلاق للبحث عن النبع السري حيث يقيم الطائر الحكيم. انطلق الأصدقاء: شموس، درة، وعطوف، مليئين بالحماس والغناء.
عند حافة الغابة الكثيفة، نبّههم عطوف إلى ضرورة التوازن بين الشجاعة والحذر، واتفقوا على البقاء معًا. دخلوا الظل الكثيف، فتلقّاهم غراب حكيم محذّرًا من المخاطر وشجّعهم على التعلم من التجربة. شكروا غرابه وتابعوا طريقهم حتى واجهوا نهرًا جارفًا. بعد تفكير جماعي، جلبوا غصنًا قويًا شكّل لهم جسرًا مؤقتًا، فنجحوا في عبوره رغم انزلاق شموس الذي أنقذه أصدقاؤه بسرعة.
حين وصلوا إلى النبع السحري، استقبلهم الطائر الحكيم بريشٍ فضي وعيون لامعة. بيّن لهم أنّ الطيران العالي لا يقتصر على قوة الأجنحة فقط، بل ينبع من نقاء القلب وصفاء النية، وأنّ التعاون والوفاء هما جناحي الارتقاء إلى سماء الأحلام. أدرك الأصدقاء أن حكمة الحكيم تعكس دروس رحلتهم.
عاد شموس ورفاقه إلى قريتهم حاملين الذكريات والعبر، وشاركوا أهل القرية تجاربهم عن العمل الجماعي والتغلب على الصعاب. أصبحت مغامرتهم مثالًا يُحتذى به، ملهمة لكل صغير وكبير للاستمرار في استكشاف عوالم جديدة بروح الوحدة والقيم النبيلة.
###
"""

In [ ]:
story="""### القصة:

 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب�

عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب

بعد أن وافق أحمد وعلي على الانضمام إلى أحمد في الرحلة، بدأا في تجهيز الأدوات اللازمة للرحلة. كان عليهما جمع بعض الماء والطعام والبوصلة وخريطة قديمة وجدها أحمد في مكتبة جده.

في صباح اليوم التالي، انطلقا في طريقهما نحو الصحراء. كان الهواء منعشًا والشمس تشرق بلطف، مما أضفى جوًا من الغموض.

عندما وصلا إلى النقطة التي حددتها الخريطة، بدأ أحمد وعلي في استكشاف الصحراء. كانت الرمال تمتد بلا نهاية تحت أقدامهم، مما أضفى شعورًا بالرهبة والإثارة على الرحلة.

بينما كانا يسيران، لاحظا شيئ

بعد ساعات من السير، لاحظا شيئًا لامعًا تحت الرمال. اقترب أحمد وعلي أكثر ليكتشفا أنه قطعة من الزجاج المكسور. كانت القطعة تبدو قديمة جدًا، لكنها تحمل نقوشًا غريبة.

قال علي وهو يشير إلى النقوش: "انظروا! إنها تبدو وكأنها جزء من خريطة!"

أجابت علي: "ربما تكون هذه بداية مغامرة جديدة."

بدأ الصديقان في فحص الخريطة بعناية، محاولين فهم الرموز والنقوش. كانت الرموز تبدو وكأنها تعليمات تقود إلى مكان معين.

قال علي وهو يشير إلى الخريطة:
 بعد أن تأكد من أن الخريطة صحيحة، عاد أحمد وعلي إلى المنزل. كانا متحمسين للغاية لرؤية ما وجداه. جلسا في غرفة المعيشة، وبدأا في سرد تفاصيل المغامرة التي تنتظرهما.

قال أحمد وهو ينظر إلى الخريطة: "هذه الخريطة تشير إلى مكان معين في الصحراء، حيث يمكن أن تكون هناك كنوز أو أسرار مخفية."

أومأ علي برأسه مؤكدًا: "علينا أن نكون حذرين، قد تكون هذه مجرد بداية لمغامرة لا تُنسى."

بدأ الصديقان في سرد القصة قائلاً: "في

###
"""

In [ ]:
def answer_question(question):

    #sys_prompt = f"أنت تلعب دور أحمد:\n{story}. مهمتك هي الإجابة على الأسئلة التي تُطرح عليك"
    sys_prompt = f"أنت تلعب دور أحمد\n. مهمتك هي الإجابة على الأسئلة التي تُطرح عليك حول القصة.\n{story}"
    #usr_prompt = f"اجب عن السؤال التالي :  {question}.\n"
    usr_prompt = f"اجب عن السؤال التالي:\n### السؤال:\n{question}\n\n### الجواب:\n"
    prompt = f"{sys_prompt}\n{usr_prompt}"
    #print ("prompt", prompt)
    # Tokenize with padding and return attention_mask
    #encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    input_ids = encoded_input.input_ids
    attention_mask = encoded_input.attention_mask
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Generate text with the attention mask explicitly provided
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        #max_length=1500,        # Adjust this if you expect longer output
        do_sample=True,        # Enable sampling for more creative text
        top_k=50,              # Top-K sampling
        top_p=0.95,            # Nucleus sampling
        num_return_sequences=1, # Generate one sequence
        #temperature=0.00001,

    )
    torch.cuda.empty_cache()
    gc.collect()

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Decode the generated tokens to text
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answer = generated_text[len(prompt):]
    #print (answer)
    torch.cuda.empty_cache()
    gc.collect()

    return answer



# Create Gradio interface for user input
with gr.Blocks() as demo:
    # Header or title for the interface
    gr.Markdown("# مولد القصة التفاعلية ")

    # Instructions or subheader
    gr.Markdown("اكتب سؤالك")

    with gr.Tabs():  # Using tabs as shown in your design

        with gr.TabItem("Question_Answer"):
            #chapter1_input = gr.Textbox(label="Chapter 1 Idea", placeholder="Describe Chapter 1 here...", lines=2)
            question_input = gr.Textbox(label="Question", placeholder="Write your question...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("Get Answer")
            # Output area to display the generated content
            output = gr.Textbox(label="Answer", interactive=False, lines=10)
            # Button click action: Capture inputs and display the generated story
            submit_button.click(answer_question, inputs= question_input, outputs=output)





# Launch the interface
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2e31773da910aed2d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2e31773da910aed2d0.gradio.live


#Feature 3 : Modify the story

In [ ]:
def modified_story(modification):

    sys_prompt = f" حسب الطلب \n{story} مهمتك هي تغيير القصة ."
    usr_prompt = f" الطلب:  {modification}.\n"
    prompt = f"{sys_prompt}\n{usr_prompt}"
    print ("prompt", prompt)
    # Tokenize with padding and return attention_mask
    #encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128)
    encoded_input = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    input_ids = encoded_input.input_ids
    attention_mask = encoded_input.attention_mask
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Generate text with the attention mask explicitly provided
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        #max_length=128,        # Adjust this if you expect longer output
        do_sample=True,        # Enable sampling for more creative text
        top_k=50,              # Top-K sampling
        top_p=0.95,            # Nucleus sampling
        num_return_sequences=1, # Generate one sequence
    )
    torch.cuda.empty_cache()
    gc.collect()

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Decode the generated tokens to text
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answer = generated_text[len(prompt):]

    torch.cuda.empty_cache()
    gc.collect()

    return answer



# Create Gradio interface for user input
with gr.Blocks() as demo:
    # Header or title for the interface
    gr.Markdown("# Interactive Story Generator")

    # Instructions or subheader
    gr.Markdown("Enter your ideas for each chapter.")

    with gr.Tabs():  # Using tabs as shown in your design

        with gr.TabItem("Modify Story"):
            #chapter1_input = gr.Textbox(label="Chapter 1 Idea", placeholder="Describe Chapter 1 here...", lines=2)
            question_input = gr.Textbox(label="Modification", placeholder="Write your modification...", lines=2)
            # Button to submit the data and generate the story
            submit_button = gr.Button("Get Answer")
            # Output area to display the generated content
            output = gr.Textbox(label="Story modified", interactive=False, lines=10)
            # Button click action: Capture inputs and display the generated story
            submit_button.click(modified_story, inputs= question_input, outputs=output)





# Launch the interface
demo.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://399e200568cc9bd1c0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


prompt  حسب الطلب 
### القصة:

 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب�

عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب

بعد أن وافق أحمد وعلي على الانضمام إلى أحمد في الرحلة، بدأا في تجهيز الأدوات اللازمة للرحلة. كان عليهما جمع بعض الماء والطعام والبوصلة وخريطة قديمة وجدها أحمد في مكتبة جده.

في صباح اليوم التالي، انطلقا في طريقهما نحو الصحراء. كان الهواء منعشًا والشمس تشرق بلطف، مما أضفى جوًا من الغموض.

عندما وصلا إلى النقطة التي حددتها

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


prompt  حسب الطلب 
### القصة:

 في قرية صغيرة تقع بين الكثبان الرملية، عاش أحمد مع أسرته في منزل بسيط تحيط به النخيل من كل جانب. كان أحمد يحب الطبيعة ويقضي ساعات طويلة في مراقبة الطيور وهي تحلق في السماء، بينما كان أحمد يستمتع بمشاهدة النجوم في السماء الصافية.

في أحد الأيام، بينما كان أحمد يلعب في الفناء الخلفي لمنزله، لاحظ شيئًا غريب�

عاد أحمد إلى المنزل وهو يحمل خريطته القديمة بعناية. كان يشعر بشيء غريب، وكأن شيئًا ما يلمع في السماء. قرر أحمد أن يأخذ خريطته إلى صديقه المقرب، علي، ليكتشفا ما وجده.

قال أحمد بحماس: "علي، انظر ماذا وجدت!"

ابتسم علي وقال: "هذه خريطة قديمة لمكان سري في الصحراء. يبدو أنها تشير إلى كنز مخفي!"

أجاب أحمد بحماس: "لكن كيف سنصل إليه؟"

أجاب

بعد أن وافق أحمد وعلي على الانضمام إلى أحمد في الرحلة، بدأا في تجهيز الأدوات اللازمة للرحلة. كان عليهما جمع بعض الماء والطعام والبوصلة وخريطة قديمة وجدها أحمد في مكتبة جده.

في صباح اليوم التالي، انطلقا في طريقهما نحو الصحراء. كان الهواء منعشًا والشمس تشرق بلطف، مما أضفى جوًا من الغموض.

عندما وصلا إلى النقطة التي حددتها